In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import random
import tensorflow as tf

_BATCH_NORM_DECAY = 0.9
_BATCH_NORM_EPSILON = 1e-5

Encoder类，输入的controller的参数，模式，以及创建好的encoder所需要的参数w_emb,参数的shape是[self.vocab_size, self.emb_size],设定的vocab_size是12，分别对应一共可能出现在x中的7个I和5个OP

目前不是很明白time_major的作用，为何要对x进行一次转置。解释如下：
time_major: The shape format of the inputs and outputs Tensors. If true, these Tensors must be shaped [max_time, batch_size, depth]. If false, these Tensors must be shaped [batch_size, max_time, depth].

tf.nn.dynamic_rnn 具体来说，设我们输入数据的格式为(batch_size, time_steps, input_size)，其中time_steps表示序列本身的长度，如在Char RNN中，长度为10的句子对应的time_steps就等于10。最后的input_size就表示输入数据单个序列单个时间维度上固有的长度。另外我们已经定义好了一个RNNCell，调用该RNNCell的call函数time_steps次，此时，得到的outputs就是time_steps步里所有的输出。它的形状为(batch_size, time_steps, cell.output_size)。state是最后一步的隐状态，它的形状为(batch_size, cell.state_size)。

build_encoder返回的是该架构对应的code (是根据encoder_ouputs通过求平均得来的)，这个code输入进mlp(predictor)然后得出预测值，predict_value，返回的还有encoder_outputs还有state表示最后一层的隐含状态

In [ ]:
class Encoder(object):
    def __init__(self, params, mode, W_emb):
        self.num_layers = params['encoder_num_layers']
        self.hidden_size = params['encoder_hidden_size']
        self.emb_size = params['encoder_emb_size']
        self.mlp_num_layers = params['mlp_num_layers']
        self.mlp_hidden_size = params['mlp_hidden_size']
        self.mlp_dropout = params['mlp_dropout']
        self.source_length = params['source_length']
        self.encoder_length = params['encoder_length']
        self.vocab_size = params['encoder_vocab_size']
        self.dropout = params['encoder_dropout']
        self.time_major = params['time_major']
        self.W_emb = W_emb
        self.mode = mode
    
    def build_encoder(self, x, batch_size, is_training):
        
        self.batch_size = batch_size
        assert x.shape.ndims == 2, '[batch_size, length]'
        x = tf.gather(self.W_emb, x)
        if self.source_length != self.encoder_length: #source_length [40,60] encoder_length=20
            tf.logging.info('Concacting source sequence along depth')
            assert self.source_length % self.encoder_length == 0
            ratio = self.source_length // self.encoder_length
            x = tf.reshape(x, [batch_size, self.source_length // ratio, ratio*self.emb_size])
        if self.time_major:
            x = tf.transpose(x, [1,0,2])
        
        cell_list = []
        for i in range(self.num_layers):
            lstm_cell = tf.contrib.rnn.LSTMCell(self.hidden_size)
            lstm_cell = tf.contrib.rnn.DropoutWrapper(lstm_cell, output_keep_prob=1-self.dropout)
            cell_list.append(lstm_cell)
        if len(cell_list) == 1:
            cell = cell_list[0]
        else:
            cell = tf.contrib.rnn.MultiRNNCell(cell_list)
        initial_state = cell.zero_state(batch_size, dtype=tf.float32)
        x, state = tf.nn.dynamic_rnn(cell, x, dtype=tf.float32, time_major=self.time_major, initial_state=initial_state)
        x = tf.nn.l2_normalize(x, dim=-1)
        self.encoder_outputs = x #(batch_size, time_steps, cell.output_size)
        self.encoder_state = state
        
        if self.time_major:
            x = tf.reduce_mean(x, axis=0)
        else:
            x = tf.reduce_mean(x, axis=1)
            
        #now, [batch_size, self.hidden_size]
        x = tf.nn.l2_normalize(x, dim=-1) 
        
        self.arch_emb = x
        
        for i in range(self.mlp_num_layers):
            name = "mlp_{}".format(i)
            x = tf.layers.dense(x, self.mlp_hidden_size, activation=tf.nn.relu, name=name)
            x = tf.layer.dropout(x, self.mlp_dropout)
            
        self.predict_value = tf.layers.dense(x, 1, activation=tf.sigmoid, name='regression')
        
        
        return {
            'arch_emb' : self.arch_emb,
            'predict_value': self.predict_value,
            'encoder_outputs': self.encoder_outputs,
            'encoder_state': self.encoder_state
        }

Model输入encoder的input和target，controller的参数，注意这里的encoder_input代表了一batch的架构

模式
训练模式，即 mode == tf.estimator.ModeKeys.TRAIN，必须提供的是 loss 和 train_op。
验证模式，即 mode == tf.estimator.ModeKeys.EVAL，必须提供的是 loss。
预测模式，即 mode == tf.estimator.ModeKeys.PREDICT，必须提供的是 predicitions。

build_graph函数用来构建整个计算图，其中包括了encoder网络架构的搭建，loss函数的计算

目前self.build_encoder()中调用了上面的Encoder类和其类函数build—encoder

self.compute_loss()也未实现

In [ ]:
class Model(object):
    def __init__(self, x, y, params, mode, scope='Encoder', reuse=tf.AUTO_REUSE):
        self.x = x
        self.y = y
        self.params = params
        self.batch_size = tf.shape(x)[0]
        self.vocab_size = params['encoder_vocab_size']
        self.emb_size = params['encoder_emb_size']
        self.hidden_size = params['encoder_hidden_size']
        self.encoder_length = params['encoder_length']
        self.weight_decay = params['weight_decay']
        self.mode = mode
        self.time_major = params['time_major']
        self.is_training = self.mode == tf.estimator.ModeKeys.TRAIN
        if not self.is_training:
            self.params['encoder_dropout'] = 0.0
            self.params['mlp_dropout'] = 0.0
        
        initializer = tf.random_uniform_initializer(-0.1,0.1)
        tf.get_variable_scope().set_initializer(initializer)
        self.build_graph(scope=scope, reuse=reuse)
    
    def build_graph(self, scope=None, reuse=tf.AUTO_REUSE):
        tf.logging.info("# creating %s graph ..." % self.mode)
        #Encoder
        with tf.variable_scope(scope, reuse=reuse):
            self.W_emb = tf.get_variable('W_emb', [self.vocab_size, self.emb_size])
            self.arch_emb, self.predict_value, self.encoder_outputs, self.encoder_state = self.build_encoder()
            if self.mode != tf.estimator.ModeKeys.PREDICT:
                self.compute_loss()
            else:
                self.loss = None
                self.total_loss = None
                
    def build_encoder(self):
        encoder = Encoder(self.params, self.mode, self.W_emb)
        res = encoder.build_encoder(self.x, self.batch_size, self.is_training)
        return res['arch_emb'], res['predict_value'], res['encoder_outputs'], res['encoder_state']
    
    def compute_loss(self):
        weights = 1 - tf.cast(tf.equal(self.y, -1.0), tf.float32)
        mean_squared_error = tf.losses.mean_squared_error(labels = self.y, predictions=self.predict_value, weights=weights)
        
        tf.identity(mean_squared_error, name='squared_error')
        tf.summary.scalar("mean_squared_error", mean_squared_error)
        
        #Add weight decay to the loss
        self.loss = mean_squared_error
        total_loss = mean_squared_error + self.weight_decay * tf.add_n([tf.nn.l2_loss(v) for v in tf.trainable_variables()])
        self.total_loss = total_loss
        
        
        
    def infer(self):
        assert self.mode == tf.estimator.ModeKeys.PREDICT
        grads_on_outputs = tf.gradients(self.predict_value, self.encoder_outputs)[0] #(batch_size, time_steps, cell.output_size)
        
        new_arch_outputs = self.encoder_outputs - self.params['predict_lambda']*grads_on_outputs
        new_arch_outputs = tf.nn.l2_normalize(new_arch_outputs, dim=-1)
        
        if self.time_major:
            new_arch_emb = tf.reduce_mean(new_arch_outputs, axis=0)
        else:
            new_arch_emb = tf.reduce_mean(new_arch_outputs, axis=1)
        
        new_arch_emb = tf.nn.l2_normalize(new_arch_emb, dim=-1)
        
        return self.arch_emb, self.predict_value, new_arch_emb, new_arch_outputs
        
        
        
        
        
        
        
        
        
        
        
        